In [11]:
import random

# Function to perform bitwise XOR
def bitwise_xor(binary_str1, binary_str2):
    assert len(binary_str1) == len(binary_str2)
    return ''.join(str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(binary_str1, binary_str2))

# Function to generate a random binary string of a given length
def generate_random_binary_string(length):
    return ''.join(str(random.randint(0, 1)) for _ in range(length))

# Function to simulate the permutation function
def permutation(X, Y):
    assert len(X) == len(Y), "Bit strings X and Y must have equal length"

    m = Y.count('1')  # Hamming weight of Y
    km_indices = [i for i in range(len(Y)) if Y[i] == '1']
    kn_indices = [i for i in range(len(Y)) if Y[i] == '0']
    n = len(X)

    # Construct the permutation Z based on the described pattern
    Z = ''.join([X[k] for k in km_indices[::-1]]) + ''.join([X[k] for k in kn_indices[::-1]])

    return Z

In [12]:
# Initialization Phase
l = 128 # The length of the values
# Step 1: Generate a l-bit random value for IDSx
IDSx = generate_random_binary_string(l)

# Step 2: Generate a l-bit random value for IDn
IDn = generate_random_binary_string(l)

# Step 3: Generate a l-bit random value for KR (temporary shared secret)
KR = generate_random_binary_string(l)

# Step 4: Generate a l-bit random value for KT (temporary shared secret)
KT = generate_random_binary_string(l)

# Step 5: Use the permutation function to create index data
index_data = {"fx(IDS1, ID1)": permutation(IDSx, IDn)}

# Define l-bit binary values for K1, K2, and K3 
K1 = generate_random_binary_string(l)
K2 = generate_random_binary_string(l)
K3 = generate_random_binary_string(l)

# Define a l-bit binary value for IDo (Identifier of the old owner)
IDo = generate_random_binary_string(l)

# Print the generated values
print("Initialization Phase:")
print("Generated IDo:", IDo)
print("Generated K1:", K1)
print("Generated K2:", K2)
print("Generated K3:", K3)
print("Generated IDSx:", IDSx)
print("Generated IDn:", IDn)
print("Generated KR:", KR)
print("Generated KT:", KT)
print("Index Data:", index_data)

Initialization Phase:
Generated IDo: 10101001101011101110101100001011010111111100000000101011000000100111110100101100100100100101001110000111011110000010010000101001
Generated K1: 00011000111100010101110110010001111001110001001010010000011100110111111101001100101000010011101001110011110100101100001000010011
Generated K2: 01000110011110001010000110111110101101011110011111100100001111101101011100010110110011001001010011001111101001001000011001001011
Generated K3: 10001111110011111101011001111011101101001110010100111011111011000110100111011100101011111110110111010110010110001100110100000001
Generated IDSx: 10010010101111111001101001010011110011010010110001000110001001110110011110000001010010110100111000110101100101110000100110111101
Generated IDn: 11100110110110110010100000010101101000011011000011000011000001011101011110111010100111100110101010111100111100111110011110001011
Generated KR: 001100001110010101010010001010110011001111110110110000101000000111001010011001100111111101110010110100

In [13]:
#%%timeit
# Disclosure attack
# The goal is to retrieve the agreed session key between the new owner and the tag

# Tag initialization (unknown to the attacker)
print("IDSx:", IDSx)
print("KT:", KT)

# Attack:
# Attacker (Generate a random value to start the attack)
A1 = generate_random_binary_string(l)  # Send "hello" and A to the tag

#Tag (calculations)
r1_prime = bitwise_xor(A1, KT)
r1_prime_xor_IDSx = bitwise_xor(r1_prime, IDSx)
B_prime = permutation(r1_prime_xor_IDSx, KT)  # Send IDSx and B' to the attacker

#print("length:", l)
KT_R = [1] * l  # The initial value of KT for recovering

# Main loop for the attack
for i in range(l):
    
  # Attacker  
  e = [0 for b in range(l)]  
  e[i] = 1  # Standard basis
  e = "".join(str(element) for element in e) 
  A = bitwise_xor(A1, e)  # Sends A and "hello" to the Tag

  # Tag  
  r = bitwise_xor(A, KT)
  r_xor_IDSx = bitwise_xor(r, IDSx)
  B = permutation(r_xor_IDSx, KT)   # Sends IDSx and B' to the attacker

  # Attacker  
  delta = bitwise_xor(B_prime, B)   
  #print("delta:", delta)  
  m = KT_R[:i].count(0) 
  if delta[(l-1)-m] == '1':
    KT_R[i] = 0
  else:
    KT_R[i] = 1 

KT_R = "".join(str(bit) for bit in KT_R)
print("KT_R (Recovered):", KT_R)
print("KT == KT_R:", KT == KT_R)

IDSx: 10010010101111111001101001010011110011010010110001000110001001110110011110000001010010110100111000110101100101110000100110111101
KT: 11111111110110000111000011000011000001010000110100000000011111010110111111010101111110110011101010101011001100101011001111011101
KT_R (Recovered): 11111111110110000111000011000011000001010000110100000000011111010110111111010101111110110011101010101011001100101011001111011101
KT == KT_R: True
